# MicroStrateg(P)ython Migration

## Load libraries

In [6]:
import json
from mstrio.object_management import folder
import pandas as pd
from time import sleep
import shutil
import os
from mstrio.connection import Connection
from mstrio.object_management.migration import (
    PackageType,
    Migration,
    PackageConfig,
    PackageSettings
)

from mstrio.object_management.migration.package import (
    Action,
    ImportStatus,
    PackageStatus,
    ValidationStatus,
)



def bld_mig_list_from_sh_fold(conn, folder_id,action= "USE_EXISTING",include_dependents= False ):

    #obj_l = get_folder_obj_l(conn=conn, folder_id=folder_id)
    i_folder = folder.Folder(connection=conn, id=folder_id)
    obj_l = i_folder.get_contents(to_dictionary=True)
    
    obj_det_l=[]
    for obj_d in obj_l:
        #print(obj_d)
        if obj_d["type"] == 18:
            obj_det_d = {}
            obj_det_d["id"] = obj_d["target_info"]["id"]
            obj_det_d["type"] = obj_d["target_info"]["type"]
            obj_det_d["action"] = action
            obj_det_d["include_dependents"] = include_dependents
            obj_det_d["subtype"] = obj_d["target_info"]["subtype"] # only for read out object definition (see end of note book)
            obj_det_l.append(obj_det_d.copy())  

    return obj_det_l


## Configure Migration

In [7]:
#define Package
usr_migration_folder_name="Daniel_Migration"
usr_package_folder_name="20250223_demo"


short_cut_folder_id="9316DEEE4820CF81E80157855D37711B"

# Connect Projects
with open('..\\config\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)

#set user credentials and open a connection to the i-server
username = user_d["username"]
password =user_d["password"]
base_url= user_d["base_url"]

source_username=username
source_password=password
source_base_url=base_url

target_username=username
target_password=password
target_base_url=base_url

source_project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
target_project_id="40DA7A1549651FD60A9D39AAB7EC77B0"
package_settings = PackageSettings(
    Action.FORCE_REPLACE,
    PackageSettings.UpdateSchema.RECAL_TABLE_LOGICAL_SIZE,
    PackageSettings.AclOnReplacingObjects.REPLACE,
    PackageSettings.AclOnNewObjects.KEEP_ACL_AS_SOURCE_OBJECT,
)

## Connect to projects

In [8]:
source_conn = Connection(base_url=source_base_url,username=source_username,
                  password=source_password,project_id=source_project_id)
#source_conn.headers['Content-type'] = "application/json"

target_conn = Connection(base_url=target_base_url,username=target_username,
                  password=target_password,project_id=target_project_id)

package_name=usr_package_folder_name
mstr_mig_folder=source_conn.environment.storage_service.location 
migration_folder= source_conn.environment.storage_service.location  +"\\"+ usr_migration_folder_name +"\\"+ usr_package_folder_name + "\\"
migration_folder


Connection to MicroStrategy Intelligence Server has been established.
Connection to MicroStrategy Intelligence Server has been established.


'D:\\shared_drive\\MSTR_Migrations\\Daniel_Migration\\20250223_demo\\'

## Create package

In [9]:
package_content_info=bld_mig_list_from_sh_fold(conn=source_conn, folder_id=short_cut_folder_id,action= "FORCE_REPLACE",include_dependents= False)
package_config = PackageConfig(
    package_settings,  package_content_info # [ package_content_from_object]
)

my_obj_mig = Migration.create_object_migration(
    connection=source_conn,
    toc_view=package_config,
    name=package_name,
    project_id=source_conn.project_id,
)

while my_obj_mig.package_info.status == PackageStatus.CREATING:
    sleep(2)
    my_obj_mig.fetch()

my_obj_mig.package_info.status

Folder object named: 'RAG_cubes' with ID: '9316DEEE4820CF81E80157855D37711B'
Successfully started creation of migration object with ID: '4A051FC5ED84445D8494346E14EC972C:3053753B23F041B380F89D420469ABAC'


PackageStatus.CREATED

In [5]:
#my_obj_mig.package_info
package_content_info

[]

## Validate Package

In [10]:
# Check migration validation status
my_obj_mig.trigger_validation(
    target_env=target_conn, target_project_name=target_conn.project_name
)

while my_obj_mig.validation.status == ValidationStatus.VALIDATING:
    sleep(2)
    my_obj_mig.fetch()


print(my_obj_mig.validation)

Successfully triggered validation process for migration with ID: '4A051FC5ED84445D8494346E14EC972C:3053753B23F041B380F89D420469ABAC'
Validation(status=ValidationStatus.VALIDATED, progress=100.0, message='Success', creation_date=datetime.datetime(2025, 3, 17, 7, 34, 16, tzinfo=datetime.timezone.utc), last_update_date=None)


## Move Migration Pack to migration folder

In [11]:
package_file = mstr_mig_folder +"/" +my_obj_mig.package_info.storage.path
package_file=os.path.normpath(package_file)
my_obj_mig_path=shutil.move(package_file, migration_folder)

print("org_file: " +package_file)
print("migration folder: " +my_obj_mig_path)

org_file: D:\shared_drive\MSTR_Migrations\mstr\shared\migrations\packages\2025-03-17\F1EDE4BCBF974828A6BCEB3ACB29ECEC.mmp
migration folder: D:\shared_drive\MSTR_Migrations\Daniel_Migration\20250223_demo\F1EDE4BCBF974828A6BCEB3ACB29ECEC.mmp



## Trigger migration

In [12]:
from_file_mig = Migration.migrate_from_file(
    connection=target_conn,
    file_path=my_obj_mig_path,
    package_type=PackageType.OBJECT,
    name="object_mig_from_local_storage",
    target_project_name=target_conn.project_name,
)

while from_file_mig.import_info.status == ImportStatus.IMPORTING:
    sleep(2)
    from_file_mig.fetch()

from_file_mig.import_info.status

Successfully started migration from file with ID: '0125EDA0FD24489CABB9A3BE62BAC4AA:3AF5053900B44AD18EC478D956C26C5C'.


ImportStatus.IMPORTED

In [16]:
from_file_mig.import_info

ImportInfo(id='1BD8215C8C034EFEB65BC616CC048618', environment={'id': 'http://85.214.60.83:8080/MicroStrategyLibrary/', 'name': 'http://85.214.60.83:8080/MicroStrategyLibrary/'}, creator=User(connection, name='Administrator', id='54F3D26011D2896560009A8E67019608'), creation_date=datetime.datetime(2025, 1, 29, 11, 23, 33, tzinfo=datetime.timezone.utc), last_updated_date=datetime.datetime(2025, 1, 29, 11, 23, 39, tzinfo=datetime.timezone.utc), status=ImportStatus.IMPORTED, import_request_status=RequestStatus.APPROVED, undo_request_status=RequestStatus.PENDING, progress=100.0, message='', undo_storage=PackageStorage(size=10912, path='mstr/shared/migrations/undo/B762307E32934B188F39FC1E6C484DD1/1BD8215C8C034EFEB65BC616CC048618.mmp'), project=Project(connection, name='MicroStrategy Tutorial PreProd', id='00F3103E41FF3743A057C3B5313595B7'), deleted=False)

## Archive Migration

### move undo mmp file to migration folder

In [17]:
## move undo mmp to migration folder
undo_source_file = mstr_mig_folder +"/" +from_file_mig.import_info.undo_storage.path
package_file=os.path.normpath(package_file)
new_undo_file=shutil.move(undo_source_file, migration_folder+"Undo.mmp")

print("org_undo_file: " +undo_source_file)
print("migration folder: " +new_undo_file)

org_undo_file: D:\shared_drive\MSTR_Migrations/mstr/shared/migrations/undo/B762307E32934B188F39FC1E6C484DD1/1BD8215C8C034EFEB65BC616CC048618.mmp
migration folder: D:\shared_drive\MSTR_Migrations\Daniel_Migration\20250223_demo\Undo.mmp


### store migrated objects to CSV file

In [18]:
package_content_df=pd.DataFrame(package_content_info)
package_content_df.to_csv(migration_folder+"package_content.csv")
package_content_df

,id,type,action,include_dependents,subtype
0,5D55D4504907E73F560BFBA95AD22A66,3,FORCE_REPLACE,False,768
1,632539C14B768DF229839BA100C92474,4,FORCE_REPLACE,False,1024
2,FBA328574EC126D4A38B51AFEF5D7856,4,FORCE_REPLACE,False,1024
3,ACB99295488C76AA566DADAE2EC30FC5,4,FORCE_REPLACE,False,1024


### save package & migration files

In [19]:
file_path=migration_folder+"package_info.txt"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(str(my_obj_mig.package_info))

file_path=migration_folder+"migration_info.txt"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(str(from_file_mig.import_info))


## Read & Store Object definition

In [20]:
from mstr_robotics.read_out_prj_obj import read_gen
i_read_gen=read_gen()

obj_def_l=[]
for obj in package_content_info:
    obj_def=i_read_gen.get_obj_def(conn=source_conn,object_id=obj["id"],obj_type=obj["type"],obj_sub_type=obj["subtype"])
    obj_def_l.append(obj_def)

file_path=migration_folder+"object_def.json"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(json.dumps(obj_def_l))
    
len(obj_def_l)

3
4
4
4


4

In [21]:
obj_def_l[0]

{'information': {'dateCreated': '2025-01-22T11:25:14.417Z',
  'dateModified': '2025-01-22T11:33:19.382Z',
  'versionId': '00508D754C04B1C3EC0C309FEB41C264',
  'acg': 255,
  'primaryLocale': 'en-US',
  'objectId': '5D55D4504907E73F560BFBA95AD22A66',
  'subType': 'report_grid',
  'name': 'Customer Detail report'},
 'sourceType': 'normal',
 'dataSource': {'dataTemplate': {'units': [{'id': '56225F324F17B61AEBD88CA256946C96',
     'name': 'Customer Address',
     'type': 'attribute',
     'nonAggregatable': False},
    {'id': '8D679D4211D3E4981000E787EC6DE8A4',
     'name': 'Item',
     'type': 'attribute',
     'forms': [{'id': '45C11FA478E745FEA08D781CEA190FE5', 'name': 'ID'}],
     'nonAggregatable': False},
    {'id': '6E069E4C11D3E4E41000E887EC6DE8A4',
     'name': 'Customer Age',
     'type': 'attribute',
     'nonAggregatable': False},
    {'id': '54BABECD11D59D57C000B28A4CC5F24F',
     'name': 'Brand',
     'type': 'attribute',
     'nonAggregatable': False},
    {'id': '8D679D3911D

In [21]:
source_conn.close()
target_conn.close()

Connection to MicroStrategy Intelligence Server has been closed.
Connection to MicroStrategy Intelligence Server has been closed.


## Run undo

In [ ]:
from_file_mig = Migration.migrate_from_file(
    connection=target_conn,
    file_path=new_undo_file,
    package_type=PackageType.OBJECT,
    name="object_mig_from_local_storage",
    target_project_name=target_conn.project_name,
)

while from_file_mig.import_info.status == ImportStatus.IMPORTING:
    sleep(2)
    from_file_mig.fetch()

from_file_mig.import_info.status